In [2]:
from IPython.display import Markdown, display

In [1]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import SummaryIndex
from llama_index.core.schema import IndexNode
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from finllmqa.api.core import LLM_API_URL

In [10]:
from llama_index.core import Settings

llm = OpenAI(model="gpt-3.5-turbo", api_base=LLM_API_URL, api_key='null')
embed_model = OpenAIEmbedding(api_base=LLM_API_URL, api_key='null')

Settings.llm = llm
Settings.embed_model = embed_model

In [14]:
from llama_index.core import ChatPromptTemplate

qa_prompt_str = (
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the question: {query_str}\n"
)

refine_prompt_str = (
    "We have the opportunity to refine the original answer "
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{context_msg}\n"
    "------------\n"
    "Given the new context, refine the original answer to better "
    "answer the question: {query_str}. "
    "If the context isn't useful, output the original answer again.\n"
    "Original Answer: {existing_answer}"
)

# Text QA Prompt
chat_text_qa_msgs = [
    (
        "system",
        "Always answer the question, even if the context isn't helpful.",
    ),
    ("user", qa_prompt_str),
]
text_qa_template = ChatPromptTemplate.from_messages(chat_text_qa_msgs)

# Refine Prompt
chat_refine_msgs = [
    (
        "system",
        "Always answer the question, even if the context isn't helpful.",
    ),
    ("user", refine_prompt_str),
]
refine_template = ChatPromptTemplate.from_messages(chat_refine_msgs)

In [26]:
query_engine = index.as_query_engine(streaming=True, 
                                     similarity_top_k=10)

In [32]:
query_engine._response_synthesizer.get_prompts()

{'text_qa_template': SelectorPromptTemplate(metadata={'prompt_type': <PromptType.QUESTION_ANSWER: 'text_qa'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings={}, function_mappings={}, default_template=PromptTemplate(metadata={'prompt_type': <PromptType.QUESTION_ANSWER: 'text_qa'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template='Context information is below.\n---------------------\n{context_str}\n---------------------\nGiven the context information and not prior knowledge, answer the query.\nQuery: {query_str}\nAnswer: '), conditionals=[(<function is_chat_model at 0x0000022D87326EF0>, ChatPromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, message_templates=[ChatMessage(role=<MessageRole.SYSTEM: 'system'>

In [28]:
response = query_engine.query(
    "根据上下文，告诉我需求价格弹性的计算公式是什么"
)
response.print_response_stream()

2024-04-08 00:08:22,680 - INFO - HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/embeddings "HTTP/1.1 200 OK"
2024-04-08 00:08:23,712 - INFO - HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/chat/completions "HTTP/1.1 200 OK"
2024-04-08 00:08:25,415 - INFO - HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/chat/completions "HTTP/1.1 200 OK"
2024-04-08 00:08:27,001 - INFO - HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/chat/completions "HTTP/1.1 200 OK"



 需求价格弹性的计算公式是：需求价格弹性 = (变化后的数量 - 变化前的数量) / (变化后的价格 - 变化前的价格)。

In [29]:
response.source_nodes[0].text

'时间的长短 物品往往随着时间变长而需求更富有弹性；当汽油价格上升时，在最初的几个月中汽油的需求量只略有减少。但是，随着时间推移，人们购买更省油的汽车，转向公共交通，或迁移到离工作地方近的地点。在几年之内，汽油的需求量会大幅度减少。\r\n计算需求价格弹性\r\n我们已经在一般意义上讨论了需求价格弹性，现在我们更精确地讨论它的计量。经济学家用需求量变动的百分比除以价格变动的百分比来计算需求价格弹性。这就是：\r\n需求价格弹性=需求量变动的百分比/价格变动的百分比\r\n例如，假定冰激凌蛋卷的价格从2美元上升到2.2美元使你购买的冰激凌从每月10个减少为8个。我们计算出价格变动百分比为：\r\n价格变动百分比＝(2.20－2.00)÷2.00×100＝10%\r\n同样，我们计算出需求量变动百分比为：\r\n需求量变动百分比＝(10－8)÷10×100＝20%\r\n在这种情况下，你的需求弹性是：需求价格弹性＝20%/10%＝2\r\n在这个例于中，弹性是2，反映了需求量变动的比例是价格变动比例的两倍。\r\n由于一种物品的需求量与其价格负相关，所以，数量变动的百分比与价格变动百分比总是相反的符号。在这个例子中，价格变动的百分比是正的10%（反映了上升），而需求量变动的百分比是负的20%（反映了减少）。由于这个原因，需求价格弹性有时称为负数。在本书中我们遵循一般做法，去掉负号，把所有价格弹性作为正数。（数学上称这种数为绝对值。）根据这个习惯，需求价格弹性越大，意味着需求量对价格越敏感。\r\n参考资料：用中点法计算弹性\r\n如果你想计算一条需求曲线上两点之间的需求价格弹性，你将很快会注意到一个令人烦恼的问题：从A点到B点的弹性似乎不同于从B点到A点的弹性。例如，考虑这些数字：\r\nA点：价格=4美元，数量=120\r\nB点：价格=6美元，数量=80\r\n从A点到B点，价格上升了50%，数量减少了33%，表明需求的价格弹性是33/50，或者0.66。与此相比，从B点到A点，价格下降了33%，而数量增加了50%，表明需求的价格弹性是50/33，或1.5。\r\n避免这个问题的一种方法是用中点法计算弹性；中点法不是用标准的方法（变动量除以原先的水平）计算变动的白分比，而是用变动量除以原先水平与最后水平的中点来计算变动的百分比。例如，4美元和6美元的中点是5美元；。

In [21]:
from llama_index.agent.openai import OpenAIAgent

# build vector index
vector_index = VectorStoreIndex.from_documents(
    docs,
)
# build summary index
summary_index = SummaryIndex.from_documents(
    docs,
)
# define query engines
vector_query_engine = vector_index.as_query_engine()
list_query_engine = summary_index.as_query_engine()

# define tools
query_engine_tools = [
    QueryEngineTool(
        query_engine=vector_query_engine,
        metadata=ToolMetadata(
            name="vector_tool",
            description=(
                f"Useful for retrieving specific context from 经济学原理"
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=list_query_engine,
        metadata=ToolMetadata(
            name="summary_tool",
            description=(
                "Useful for summarization questions related to 经济学原理"
            ),
        ),
    ),
]

# build agent
function_llm = OpenAI(temperature=0, model="gpt-3.5-turbo", api_base=LLM_API_URL, api_key='null')
agent = OpenAIAgent.from_tools(
    query_engine_tools,
    llm=function_llm,
    verbose=True,
)

INFO:httpx:HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/embeddings "HTTP/1.1 200 OK"
HTTP Reque

In [22]:
# define top-level nodes
book_title = '经济学原理(微观经济学)'
book_summary = (
    f"This content contains content about {book_title}. Use"
    " this index if you need to lookup specific facts about"
    f" {book_title}.\n"
)
node = IndexNode(
    text=book_summary, index_id=book_title, obj=agent
)

In [23]:
# define top-level retriever
vector_index = VectorStoreIndex(
    objects=[node],
)
query_engine = vector_index.as_query_engine(similarity_top_k=1, verbose=True)

INFO:httpx:HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/embeddings "HTTP/1.1 200 OK"


In [24]:
# should use Boston agent -> vector tool
response = query_engine.query("什么是经济学十大原理")

INFO:httpx:HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/embeddings "HTTP/1.1 200 OK"
Retrieval entering 经济学原理(微观经济学): OpenAIAgent
Retrieving from object OpenAIAgent with query 什么是经济学十大原理
Added user message to memory: 什么是经济学十大原理
INFO:httpx:HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/chat/completions "HTTP/1.1 200 OK"


In [34]:
from finllmqa.agent.langchain_tools import KGRetrieverTool

In [35]:
tool = KGRetrieverTool()

In [38]:
tool.graph_searcher.search_main({'主体': {'股票': ['贵州茅台']}, '时间': ['2024年'], '意图': ['市净率']})

'*贵州茅台2024-06-30的市净率信息如下\n存在市净率如下表所示\n|属性|值|\n|name|2024-06-30|\n|报告期|2024-06-30|\n|股票代码|600519|\n\n'

In [ ]:
from llama_index.graph_stores.nebula import NebulaGraphStore

In [2]:
import json
with open('../FinEval/subject_mapping.json', 'r') as f:
    subject_mapping = json.load(f)
subjects = subject_mapping.keys()

In [14]:
list(subjects)[30:35]

['insurance', 'international_finance', 'investments', 'monetary_finance']

In [9]:
['banking_practitioner_qualification_certificate', 'fund_qualification_certificate', 'futures_practitioner_qualification_certificate',
 'securities_practitioner_qualification_certificate','statistics', 'financial_engineering', 'investments', 'monetary_finance']

34